#Cluster

#UTS

In [32]:
!pip install swifter
!pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
import swifter
url = 'https://raw.githubusercontent.com/baihakibai/prosdat/main/dataset-baru.csv'
df = pd.read_csv(url)
text = df["Abstrak"]
text

0      Sistem  informasi  akademik  (SIAKAD) merupaka...
1      Berjalannya koneksi jaringan komputer dengan l...
2      Web server adalah sebuah perangkat lunak serve...
3      Penjadwalan  kuliah  di  Perguruan  Tinggi  me...
4      Seiring perkembangan teknologi yang ada diduni...
                             ...                        
800    Investasi saham selama ini memiliki resiko ker...
801    Information Retrieval (IR) merupakan pengambil...
802    Klasifikasi citra merupakan proses pengelompok...
803    Identifikasi atribut pejalan kaki merupakan sa...
804    Topik deteksi objek telah menarik perhatian ya...
Name: Abstrak, Length: 805, dtype: object

In [34]:
def case_folding(comment):
    comment = comment.lower()
    return comment
text = text.apply(case_folding)
text

0      sistem  informasi  akademik  (siakad) merupaka...
1      berjalannya koneksi jaringan komputer dengan l...
2      web server adalah sebuah perangkat lunak serve...
3      penjadwalan  kuliah  di  perguruan  tinggi  me...
4      seiring perkembangan teknologi yang ada diduni...
                             ...                        
800    investasi saham selama ini memiliki resiko ker...
801    information retrieval (ir) merupakan pengambil...
802    klasifikasi citra merupakan proses pengelompok...
803    identifikasi atribut pejalan kaki merupakan sa...
804    topik deteksi objek telah menarik perhatian ya...
Name: Abstrak, Length: 805, dtype: object

In [35]:
def linkNormalize(text):
    text = re.sub(r"\s—\s", "", text)
    text = re.sub(r"http\S+", "", text)
    return text
text = text.apply(linkNormalize)
text

0      sistem  informasi  akademik  (siakad) merupaka...
1      berjalannya koneksi jaringan komputer dengan l...
2      web server adalah sebuah perangkat lunak serve...
3      penjadwalan  kuliah  di  perguruan  tinggi  me...
4      seiring perkembangan teknologi yang ada diduni...
                             ...                        
800    investasi saham selama ini memiliki resiko ker...
801    information retrieval (ir) merupakan pengambil...
802    klasifikasi citra merupakan proses pengelompok...
803    identifikasi atribut pejalan kaki merupakan sa...
804    topik deteksi objek telah menarik perhatian ya...
Name: Abstrak, Length: 805, dtype: object

In [36]:
symbols = ",!\"#$%&()*+-.…/:;<=>?@[\]^_`{|}~\n0987654321"
def cleaning_text(comment):
    for i in symbols:
        comment = np.char.replace(comment, i, ' ')
    return comment
text = text.apply(cleaning_text)
text

0      sistem  informasi  akademik   siakad  merupaka...
1      berjalannya koneksi jaringan komputer dengan l...
2      web server adalah sebuah perangkat lunak serve...
3      penjadwalan  kuliah  di  perguruan  tinggi  me...
4      seiring perkembangan teknologi yang ada diduni...
                             ...                        
800    investasi saham selama ini memiliki resiko ker...
801    information retrieval  ir  merupakan pengambil...
802    klasifikasi citra merupakan proses pengelompok...
803    identifikasi atribut pejalan kaki merupakan sa...
804    topik deteksi objek telah menarik perhatian ya...
Name: Abstrak, Length: 805, dtype: object

In [37]:
nltk.download('stopwords')
list_stopwords = stopwords.words('indonesian')
def stopword_removal(comment):
    comment = comment.split()
    data = [word for word in comment if word not in list_stopwords]
    kalimat = ""
    for i in data:
      kalimat += i
      kalimat += " "
    return kalimat


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()
def stemmingNormalize(comment):
    comment = comment.split()
    data = []
    for term in comment:
        term = stemmer.stem(term)
        data.append(term)
    kalimat = ""
    for i in data:
      kalimat += i
      kalimat += " "
    return kalimat
text = text.swifter.apply(stemmingNormalize)
text

Pandas Apply:   0%|          | 0/805 [00:00<?, ?it/s]

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.get_feature_names_out

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_separate = tfidf_vectorizer.fit_transform(text)

df_tfidf = pd.DataFrame(
    tfidf_separate.toarray(), columns=tfidf_vectorizer.get_feature_names_out(), index=df.index
)
X = df_tfidf.values
df_tfidf

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components =10)

X = pca.fit_transform(X)
# X_test = pca.transform(X_test)

pca.explained_variance_ratio_

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X)

In [ ]:
center = kmeans.cluster_centers_
center

In [ ]:
y_pred = kmeans.labels_


data = kmeans.labels_
res = []
for i in data:
    if i == 0:
        res.append("RPL")
    else:
        res.append("KOMPUTASI")
print(res)

In [ ]:
# df['Kategori'].replace("RPL",1, inplace=True)
# df['Kategori'].replace("KOMPUTASI",0, inplace=True)
y_true = df['Kategori'].values.tolist()
print(df['Kategori'].values.tolist())

In [ ]:
# from sklearn.metrics import accuracy_score
# print ("Accuracy : ",
#     accuracy_score(y_true, y_pred)*100)

In [ ]:
# df = pd.read_csv("dataset-baru.csv")
# df['Kategori'].replace("RPL",0, inplace=True)
# df['Kategori'].replace("KOMPUTASI",1, inplace=True)
# y_true = df['Kategori'].values.tolist()
# print(df['Kategori'].values.tolist())

In [ ]:
# from sklearn.metrics import accuracy_score
# print ("Accuracy : ",
#     accuracy_score(y_true, y_pred)*100)

In [ ]:
n = len(y_true)
for i in range(n):
    print(res[i],"\t : ",y_true[i])

In [ ]:
from sklearn.metrics import accuracy_score
print ("Accuracy : ",
    accuracy_score(y_true, res)*100)